In [1]:
# imports and link xml
import re , folium , pandas as pd , xml.etree.ElementTree as ET
from math import radians, cos, sin, asin, sqrt
from IPython.display import display

tree = ET.parse('session/Kismet-20191011-18-31-55-1.netxml')
gps_tree = ET.parse('session/Kismet-20191011-18-31-55-1.gpsxml')
root , gps_root = tree.getroot() , gps_tree.getroot()

In [2]:
# find vendor with MAC address
file = open('mac-vendors.txt', 'r', encoding="utf8")
mac_dict = { mac[0]: mac[1] for mac in [re.split(r'\t+', line.strip()) for line in file] }
def getDeviceType(bssid):
    return mac_dict.get(bssid[:8])

In [3]:
def haversine(lon1, lat1, lon2, lat2): # haversine formula for point distance
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon , dlat = lon2 - lon1 , lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    m = 6371* c/ 1.069
    return m

In [4]:
def appendArray(net, child): # parse xml and create array data
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], []
        if (net == "wireless-network"): scan = 'BSSID' 
        else: scan = 'client-mac'        
        for bssid_tag in child.findall(scan):
            bssid = bssid_tag.text 
        for gps_tag in child.findall("gps-info"):
            gps = [[gps_tag[0].text, gps_tag[1].text],[gps_tag[4].text, gps_tag[5].text]]
        time = [child.attrib['first-time'], child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'):
                essid= essid_tag.text
            encryption = [info_tag.text for info_tag in ssid_tag if info_tag.tag == "encryption"]
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid]) 

In [18]:
StationList = [] # MAC, [[min GPS],[max GPS]], [[first seen],[last seen]], device type, encryption, ESSID
for child in root:
    appendArray("wireless-network",child)
    for subchild in child:
        appendArray("wireless-client", subchild)

### creep detecting starts here

In [19]:
creeps = [] # list of creeps
for index in range(0,len(StationList)):
    distance = haversine(float(StationList[index][1][0][0]),float(StationList[index][1][0][1]),float(StationList[index][1][1][0]),float(StationList[index][1][1][1]))
    if (distance > .20): # .20 mile threshold
        creeps.append(StationList[index])
        
# filter duplicates from Kismet output
unique, duplicate = [], []
for creep, i in zip(creeps, range(len(creeps))) :
    if creep[0] not in unique:
        unique.append(creep[0])
    else:
        duplicate.append(i)
for index in sorted(duplicate, reverse=True):
    del creeps[index]

for child in creeps :
    child[1] = []
    for gpss in gps_root:
        if (bool(gpss.attrib) and gpss.attrib['bssid']==child[0]):
            child[1].append([float(gpss.attrib["lat"]),float(gpss.attrib["lon"])])
# list of all creeps
pd.DataFrame(creeps, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

,BSSID,GPS,Dates,Manufacturer,Encryption,ESSID
0,00:0D:97:00:5B:33,"[[34.196182, -118.340149], [34.196323, -118.34...","[Fri Oct 11 18:35:27 2019, Fri Oct 11 18:45:34...",ABB Inc./Tropos,[None],BWP Free WiFi
1,00:0D:97:09:B1:A4,"[[34.19574, -118.34005], [34.190941, -118.3417...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:45:43...",ABB Inc./Tropos,[None],None
2,10:DA:43:C7:32:ED,"[[34.193405, -118.340103], [34.193405, -118.34...","[Fri Oct 11 18:37:01 2019, Fri Oct 11 18:43:06...",NETGEAR,"[WPA+PSK, WPA+AES-CCM]",A-BEK-5G
3,00:01:5C:A6:8A:46,[],"[Fri Oct 11 18:37:01 2019, Fri Oct 11 18:43:06...",CADANT INC.,[],
4,8C:45:00:0C:F2:95,"[[34.196125, -118.34024], [34.196316, -118.340...","[Fri Oct 11 18:32:48 2019, Fri Oct 11 18:45:39...",None,[],
5,B0:72:BF:EE:15:42,"[[34.196342, -118.340179], [34.196342, -118.34...","[Fri Oct 11 18:32:28 2019, Fri Oct 11 18:43:30...","Murata Manufacturing Co., Ltd.",[],
6,B6:E6:2D:53:20:76,"[[34.196041, -118.340248], [34.195942, -118.34...","[Fri Oct 11 18:33:13 2019, Fri Oct 11 18:45:24...",None,"[WPA+TKIP, WPA+PSK, WPA+AES-CCM]",Creeper


In [20]:
'''
this code tries its best to approximately draw your path
'''
coordinates = list(set(map(tuple,[[float(child.attrib['lat']),float(child.attrib['lon'])] for child in gps_root if child.tag == "gps-point"])))
def findClose(loop) :
    for iter in range(0,loop):
        for coord in range(0, len(coords)):
            low = 10000000000000000
            for nextc in range(coord+1, len(coords)):
                if (haversine(coords[coord][0],coords[coord][1],coords[nextc][0],coords[nextc][1]) < low):
                    low = haversine(coords[coord][0],coords[coord][1],coords[nextc][0],coords[nextc][1])
                    index = nextc
            coords.insert(coord+1, coords.pop(index))
        coords.reverse()
    
from copy import deepcopy
coords = deepcopy(coordinates)
print(len(coords))
index = 0
findClose(2)
    
large = 0
for i in range(0, len(coords)-1):
    if haversine(coords[i][0],coords[i][1],coords[i+1][0],coords[i+1][1])>large:
        large= haversine(coords[i][0],coords[i][1],coords[i+1][0],coords[i+1][1])
        index = i
coords.insert(0, coords.pop(index))
findClose(3)

679


In [29]:
from folium import FeatureGroup, LayerControl, Map, Marker
LDN_COORDINATES = (34.196136, -118.340164)
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=30) 

folium.PolyLine(coords,line_opacity = 0.5).add_to(myMap)

creep1 = FeatureGroup(name='Creep 1')
creep2 = FeatureGroup(name='Creep 2')
creep3 = FeatureGroup(name='Creep 3')
creep4 = FeatureGroup(name='Creep 4')
creep5 = FeatureGroup(name='Creep 5')
creep6 = FeatureGroup(name='Creep 6')

for creep_loc in creeps[0][1]:
    Marker(location=creep_loc, popup='Creep 1').add_to(creep1)
for creep_loc in creeps[1][1]:
    Marker(location=creep_loc, popup='Creep 2').add_to(creep2)
for creep_loc in creeps[2][1]:
    Marker(location=creep_loc, popup='Creep 3').add_to(creep3)
for creep_loc in creeps[3][1]:
    Marker(location=creep_loc, popup='Creep 4').add_to(creep4)
for creep_loc in creeps[4][1]:
    Marker(location=creep_loc, popup='Creep 5').add_to(creep5)
for creep_loc in creeps[5][1]:
    Marker(location=creep_loc, popup='Creep 6').add_to(creep6)
    
creep1.add_to(myMap)
creep2.add_to(myMap)
creep3.add_to(myMap)
creep4.add_to(myMap)
creep5.add_to(myMap)
creep6.add_to(myMap)
LayerControl().add_to(myMap)

display(myMap)